Does it get better or worse?

Presence of elements in ISO/FGDC:

- data quality
- data quality with lineage
- attribute definitions
- distribution information
- metadata reference section

Some of the above will include a word count value extracted from certain elements. For data quality, the count will come from the quality descriptions excluding lineage. The lineage word count will be included separately based on the process step descriptions. Attribute word counts will be taken for FGDC only, and from the main description plus any attribute descriptions.

In [ ]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import *
from sqlalchemy import and_
from semproc.xml_utils import *
from mpp.models import Response
from datetime import datetime
from lxml import etree
import os

In [ ]:
# fgdc xpath sets for text extraction

# for data quality sans lineage
fgdc_dq_xpaths = [
    ['dataqual', 'attracc', 'attraccr'],
    ['dataqual', 'attracc', 'qattracc', 'attracce'],
    ['dataqual', 'logic'],
    ['dataqual', 'complete'],
    ['dataqual', 'posacc', 'horizpa', 'horizpar'],
    ['dataqual', 'posacc', 'horizpa', 'qhorizpa', 'horizpae'],
    ['dataqual', 'posacc', 'vertacc', 'vertaccr'],
    ['dataqual', 'posacc', 'vertacc', 'qhorizpa', 'vertacce']
]

# for data quality lineage
fgdc_lineage_xpaths = [
    ['dataqual', 'lineage', 'procstep', 'procdesc']
]

# for attributes
fgdc_attr_xpaths = [
    ['eainfo', 'overview'],
    ['eainfo', 'eadetcit'],
    ['eainfo', 'detailed', 'attr', 'attrdef'],
    ['eainfo', 'detailed', 'attr', 'attrlabl']
]

# checks just for a decent existence
# mandatory doesn't mean they exist :/
fgdc_existences = [
    ('dataqual', 'dataqual/logic or dataqual/complete'),
    ('lineage', 'dataqual/lineage/procstep'),
    ('attributes', 'eainfo/detailed/attrdef or eainfo/detailed/attrlabl'),
    ('metadata ref', 'metainfo/metstdn')  # just check for the standard name
]

In [ ]:
# iso xpath sets

# for data quality sans lineage
iso_dq_xpaths = [
    
]

# for data quality lineage
iso_lineage_xpaths = [
    
]

# for attributes
iso_attr_xpaths = [
    
]

iso_existences = [
    ()
]

In [ ]:
# return a dict of fq xpaths: text from one of our sets
# in this case, we aren't interested in element attributes
# or in iterating over each child, just elements where
# there's an expectation (based on cultural practices)
# of finding descriptive text.
def extract(xml, xpath):
    elems = extract_elems(xml, xpath)
    for elem in elems:
        text = elem.text if elem.text else ''
        if not text:
            continue

        # xpath definition doesn't necessarily include
        # every elem name from parent, so return exact path
        tags = '/'.join(_taggify(elem))
        yield (tags, text.strip())

def _extract_tag(t):
    if not t:
        return
    return t.split('}')[-1]

def _taggify(e):
    tags = [e.tag] + [m.tag for m in e.iterancestors()]
    tags.reverse()

    try:
        return [_extract_tag(t) for t in tags]
    except:
        return []

In [ ]:
# load the postgres connection file
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [ ]:
sketchy_sql = '''with i
as (
    select d.response_id, jsonb_array_elements(d.identity::jsonb) ident
    from identities d
    where d.identity is not null
)

select r.id, r.source_url, r.source_url_sha, r.cleaned_content, i.ident->'protocol' as protocol
from responses r join i on i.response_id = r.id
where i.ident->>'protocol' = 'FGDC' or i.ident->>'protocol' = 'ISO'
limit %s
offset %s;
'''



In [ ]:
LIMIT=500
for i in xrange(0, 46000, LIMIT):
    sql = sketchy_sql % (LIMIT, i)
    result = session.execute(sql)
    for r in result:
        if os.path.exists('outputs/metrics/%s.json' % r['id']):
            continue
            
        try:
            xml = etree.fromstring(r['cleaned_content'].encode('utf-8'))
        except:
            print 'xml fail', r['id']
            continue
            
        metrics = {
            "data_quality": False,
            "data_quality_bow": 0,
            "lineage": False,
            "lineage_bow": 0,
            "attribute_ref": False,
            "attribute_bow": 0
            "metadata_ref": False,
            "distribution": False
        }

        if r['protocol'] == 'ISO':
            # 
        elif r['protocol'] == 'FGDC':
            dataqual_elem = extract_elem(xml, ['dataqual'])
            if dataqual_elem is not None:
                
            
            # data quality
            
            # dataqual lineage
            
            # eainfo
            
            # metadata ref?